In [1]:
import sys
import os
import importlib
sys.path.append('..')

import utils
from utils.dataloader import Dataloader
from utils.extract import extract_fields
from utils.rule_gen import generate_query
from utils.evaluator import evaluate_output

from ipywidgets import Layout
import hyperwidget

In [2]:
def get_dataloader():
    w2_sample_dir = os.path.join('../data', 'sample', 'w2')
    data_dir = os.path.join(w2_sample_dir, 'single_clean')
    label_path = os.path.join(w2_sample_dir, 'single_label.csv')
    return Dataloader(data_dir, label_path)

dl = get_dataloader()

In [3]:
field_queries = [
    {   # Field 1
        "name": "EIN",
        "arguments": {
            "x-position": 0.1,
            "y-position": 0.1,
            "entity": "EIN",
            "word-neighbors": ["Employer", "Identification", "Number"],
            "word-neighbor-top-thres": 0.05,
            "word-neighbor-left-thres": 0.1,
        },
        "weights": {
            "x-position": 0.25,
            "y-position": 0.25,
            "entity": 0.25,
            "word-neighbors": 0.25,
        }
    },
    {   # Field 2
        "name": "Medicare Tax withheld",
        "arguments": {
            "x-position": 0.9,
            "y-position": 0.1,
            "entity": "CARDINAL",
            "word-neighbors": ["Medicare", "Tax", "Withheld"],
            "word-neighbor-top-thres": 0.05,
            "word-neighbor-left-thres": 0.1,
        },
        "weights": {
            "x-position": 0.25,
            "y-position": 0.25,
            "entity": 0.25,
            "word-neighbors": 0.25,
        }
    }
]

In [4]:
k = 100
num_docs = len(dl)

extracted_fields = [
    extract_fields(dl.get_document(i), field_queries, k)
    for i in range(num_docs)
]
labels = [dl.get_label(i) for i in range(num_docs)]

In [5]:
errors = evaluate_output(extracted_fields, labels, ['EIN', 'Medicare Tax withheld'])

Field: EIN	Accuracy: 1.0
Field: Medicare Tax withheld	Accuracy: 1.0


In [6]:
error_table = hyperwidget.ErrorTable(
    errors = errors
)

In [7]:
pages = [dl.get_document(i).pages[0].as_dict() for i in range(num_docs)]
labels = [dl.get_label(i) for i in range(num_docs)]
extracted_fields_serializable = [
    {
        k: [f.as_dict() for f in fl]
        for (k, fl)in fd.items()
    }
    for fd in extracted_fields
]

extraction_heatmap = hyperwidget.ExtractionHeatmap(
    pages=pages,
    labels=labels,
    extracted_fields=extracted_fields_serializable
)

In [8]:
error_table

ErrorTable(errors={'EIN': [], 'Medicare Tax withheld': []})

In [9]:
extraction_heatmap

ExtractionHeatmap(extracted_fields=[{'EIN': [{'line': {'height': 28, 'width': 136, 'left': 66, 'top': 76, 'tex…

In [10]:
ocr_visualizer = hyperwidget.OCRVisualizer(
    page=dl.get_document(0).pages[0].as_dict()
)

In [11]:
ocr_visualizer

OCRVisualizer(page={'width': 1236, 'height': 1658, 'b64_image': 'iVBORw0KGgoAAAANSUhEUgAABNQAAAZ6CAIAAACbo6xpA…

In [12]:
print("Chosen Lines: ", [label_page.lines[i] for i in ocr_visualizer.line_idxs])
query = generate_query("Control number", label_page.lines[ocr_visualizer.line_idxs[0]], label_page)
print("Generated Query: ", query)

Chosen Lines:  []


NameError: name 'label_page' is not defined